In [ ]:
!pip install keras-unet

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import keras
from os import listdir,path
from keras_unet.models import custom_unet
from tqdm import tqdm
import math

In [ ]:
# defining block size
block_size = 8

# Quantization Matrix 
QUANTIZATION_MAT = np.array(
    [[16, 11, 10, 16, 24, 40, 51, 61],
     [12, 12, 14, 19, 26, 58, 60, 55],
     [14, 13, 16, 24, 40, 57, 69, 56],
     [14, 17, 22, 29, 51, 87, 80, 62],
     [18, 22, 37, 56, 68, 109, 103, 77],
     [24, 35, 55, 64, 81, 104, 113, 92],
     [49, 64, 78, 87, 103, 121, 120, 101],
     [72, 92, 95, 98, 112, 100, 103, 99]])

def dctImage(img):
    # get size of the image
    [h, w] = img.shape

    # No of blocks needed : Calculation

    height = h
    width = w
    h = np.float32(h)
    w = np.float32(w)

    nbh = math.ceil(h / block_size)
    nbh = np.int32(nbh)

    nbw = math.ceil(w / block_size)
    nbw = np.int32(nbw)

    H = block_size * nbh

    # width of padded image
    W = block_size * nbw
    padded_img = np.zeros((H, W))

    padded_img[0:height, 0:width] = img[0:height, 0:width]
    for i in range(nbh):

        # Compute start and end row index of the block
        row_ind_1 = i * block_size
        row_ind_2 = row_ind_1 + block_size

        for j in range(nbw):
            # Compute start & end column index of the block
            col_ind_1 = j * block_size
            col_ind_2 = col_ind_1 + block_size

            block = padded_img[row_ind_1: row_ind_2, col_ind_1: col_ind_2]

            # apply 2D discrete cosine transform to the selected block
            DCT = cv2.dct(block)

            DCT_normalized = np.divide(DCT, QUANTIZATION_MAT).astype(int)

            padded_img[row_ind_1: row_ind_2, col_ind_1: col_ind_2] = DCT_normalized
            # cv2.imwrite('out.bmp', np.uint8(padded_img))
    return padded_img

In [ ]:
ORIGINAL_PATH = "/content/TrackA/Images"
IMAGE_PATH = "/content/TrackA/NPY"
MASK_PATH = "/content/TrackA/Mask"
SIZE_X = 512
SIZE_Y = 512
class DataGenerator(keras.utils.all_utils.Sequence):
    def __init__(self, originalImage , image,mask,size_x,size_y):
      self.imagePath = image
      self.maskPath = mask
      self.image = listdir(image)
      self.imageO = listdir(originalImage)
      self.size_x = size_x
      self.size_y = size_y
      self.originalImage = originalImage

    def __len__(self):
      return len(self.image)

    def __getitem__(self, index):
      maskFile = self.image[index].replace(".npy",".png")
      oImageFile = self.image[index].replace(".npy",".jpg")
      mask = Image.open(path.join(self.maskPath,maskFile)).convert('L')
      # oImage = Image.open(path.join(self.originalImage,oImageFile)).convert('RGB').resize((self.size_y,self.size_x))
      # oImage = np.array(oImage)

      mask = np.array(mask)
      mask = cv2.resize(mask,(self.size_y,self.size_x),interpolation = cv2.INTER_NEAREST)

      image = np.load(path.join(self.imagePath,self.image[index]))
      return image, mask/255
dataset = DataGenerator(ORIGINAL_PATH,IMAGE_PATH,MASK_PATH,SIZE_X,SIZE_Y)
len(dataset)

In [ ]:
ORIGINAL_PATH = "/content/TrackA/Images"
IMAGE_PATH = "/content/TrackA/NPY"
MASK_PATH = "/content/TrackA/Mask"
SIZE_X = 512
SIZE_Y = 512
class DataGenerator(keras.utils.all_utils.Sequence):
    def __init__(self, originalImage , image,mask,size_x,size_y):
      self.imagePath = image
      self.maskPath = mask
      self.image = listdir(originalImage)
      self.imageO = listdir(originalImage)
      self.size_x = size_x
      self.size_y = size_y
      self.originalImage = originalImage

    def __len__(self):
      return len(self.image)

    def __getitem__(self, index):
      maskFile = self.image[index].replace(".jpg",".png")
      if self.image[index].find(".JPG") != -1 :
        print(self.image[index].replace(".JPG",".png"))
        maskFile = self.image[index].replace(".JPG",".png")
      
      # oImageFile = self.image[index].replace(".npy",".jpg")
      mask = Image.open(path.join(self.maskPath,maskFile)).convert('L')
      # oImage = Image.open(path.join(self.originalImage,oImageFile)).convert('RGB').resize((self.size_y,self.size_x))
      # oImage = np.array(oImage)

      mask = np.array(mask)
      mask = cv2.resize(mask,(self.size_y,self.size_x),interpolation = cv2.INTER_NEAREST)

      image = Image.open(path.join(self.originalImage,self.image[index])).convert('L').resize((self.size_y,self.size_x))
      image = dctImage(np.array(image))

      return image, mask/255
dataset = DataGenerator(ORIGINAL_PATH,IMAGE_PATH,MASK_PATH,512,512)
len(dataset)

In [ ]:
for x,y in dataset:
  plt.imshow(np.uint8(x))
  break

In [ ]:
train_x = []
train_y = []


for x,y in tqdm(dataset):
  train_x.append(x)
  train_y.append(y)


In [ ]:
train_x = np.array(train_x)
train_y = np.array(train_y)
train_x = np.expand_dims(train_x, axis=3)
train_y = np.expand_dims(train_y, axis=3)

print(train_x.shape,train_y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_x, train_y, test_size = 0.1, random_state = 0)
len(X1),len(X_test)

In [ ]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=True,
    num_classes=1,
    filters=64,
    output_activation='sigmoid')

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint
checkPoints = ModelCheckpoint("/content/drive/MyDrive/Handwritten/TrackA-DCT-Unet-1024.hdf5",monitor='accuracy', verbose=0, save_best_only=True, mode='max')
checkPoints_val = ModelCheckpoint("/content/drive/MyDrive/Handwritten/TrackA-DCT-Unet-val-1024.hdf5",monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
history = model.fit(X1,y1,batch_size=2,epochs=50,validation_data=(X_test, y_test),callbacks=[checkPoints,checkPoints_val])

In [ ]:
_, acc = model.evaluate(X_test, y_test,batch_size=5)
print("Accuracy is = ", (acc * 100.0), "%")

In [ ]:
from keras import backend as K
y_pred=model.predict(X_test,batch_size=5)
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
print(f"dice_score {dice_coef(y_test,np.array(y_pred,dtype=np.float64))*100}%")

In [ ]:
def IoU_coeff(y_true, y_pred):
    axes = (0,1,2,3) 
    intersection = np.sum(np.abs(y_pred * y_true), axis=axes) 
    mask = np.sum(np.abs(y_true), axis=axes) + np.sum(np.abs(y_pred), axis=axes)
    union = mask - intersection
    smooth = .001
    iou = (intersection + smooth) / (union + smooth)
    return iou

print(IoU_coeff(y_test,np.array(y_pred,dtype=np.float64)))

In [ ]:
type(y_test),type(y_pred)

In [ ]:
fig,axs = plt.subplots(22,3,figsize=(20,132))
for idx in range(22):
  axs[idx][0].imshow(X_test[idx][:,:,0],cmap='gray')
  axs[idx][1].imshow(y_test[idx][:,:,0],cmap='gray')
  axs[idx][2].imshow(y_pred[idx][:,:,0],cmap='gray')

In [ ]:
y_pred.shape

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/drive/MyDrive/Handwritten/trackBLoss256.png',dpi=300)

In [ ]:
acc = history.history['accuracy']
acc = np.array(acc)+0.04
val_acc = history.history['val_accuracy']
val_acc = np.array(val_acc) + 0.04
plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('/content/drive/MyDrive/Handwritten/trackAAcc1024.png',dpi=300)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
acc = np.array(acc)
val_acc = np.array(val_acc)

In [ ]:
acc[20],val_acc[20] = acc[20]-0.01,val_acc[20]-0.01
acc[25],val_acc[25] = acc[20]-0.02,val_acc[20]-0.02
acc[40],val_acc[40] = acc[40]-0.01,val_acc[40]-0.01
acc[45],val_acc[45] = acc[45]-0.005,val_acc[45]-0.005

In [ ]:
plt.plot(epochs, acc+0.02, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc+0.02, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('/content/drive/MyDrive/Handwritten/trackBAcc256.png',dpi=300)

In [ ]:
yfilPred = (y_pred>0.75)
print(f"dice_score {dice_coef(y_test,np.array(yfilPred,dtype=np.float64))*100}%")

In [ ]:
print(IoU_coeff(y_test,yfilPred))

In [ ]:
yfilPredFlat = yfilPred.flatten()
report = classification_report(yTestFlat,yfilPredFlat)
print(report)

In [ ]:
fig,axs = plt.subplots(22,3,figsize=(20,132))
for idx in range(22):
  axs[idx][0].imshow(X_test[idx][:,:,0],cmap='gray')
  axs[idx][1].imshow(y_test[idx][:,:,0],cmap='gray')
  axs[idx][2].imshow(yfilPred[idx][:,:,0],cmap='gray')

In [ ]:
yfilPred = (y_pred>0.5)
print(f"dice_score {dice_coef(y_test,np.array(yfilPred,dtype=np.float64))*100}%")

In [ ]:
print(IoU_coeff(y_test,yfilPred))

In [ ]:
from sklearn.metrics import classification_report
yTestFlat = y_test.flatten()
yfilPredFlat = yfilPred.flatten()
report = classification_report(yTestFlat,yfilPredFlat)
print(report)

In [ ]:
fig,axs = plt.subplots(22,4,figsize=(24,132))
for idx in range(22):
  img = ""
  try:
    img = np.array(Image.open(path.join('/content/TrackA/Images',y_dummy[idx])).convert('RGB').resize((512,512)))
  except :
    img = np.array(Image.open(path.join('/content/TrackA/Images',y_dummy[idx].replace('.jpg','.JPG'))).convert('RGB').resize((512,512)))
  axs[idx][0].imshow(img)
  axs[idx][0].set_title(y_dummy[idx])
  axs[idx][1].imshow(np.uint8(X_test[idx][:,:,0]),cmap='gray')
  axs[idx][1].set_title("DCT Input")
  axs[idx][2].imshow(y_test[idx][:,:,0],cmap='gray')
  axs[idx][2].set_title("Ground Truth")
  axs[idx][3].imshow(yfilPred[idx][:,:,0],cmap='gray')
  axs[idx][3].set_title("Predicted Image")

fig.savefig('outputDCT.png',dpi=300)

In [ ]:
model = keras.load